In [91]:
import csv
import pprint
import pandas as pd
import pywikibot
import re
from string import punctuation

from caphi2ipa import caphipp2ipa

# To do

* Singular nouns:
    * Add plurals
    * Handle "complex" nouns:
        * Several glosses (';')
        * Complex glosses ('_' or ' ')
    * Add IPA
* Plural nouns
* Adjectives
* Active participles (NOUN_ACT:MS)
* Passive participles (NOUN_PASS)
* INTERJ
* ADV
* NOUN:PHRASE
* Verbs
* VERB:PHRASE
* Issues:
    * شرعية: WARNING: API warning (edit): The value passed for "text" contains invalid or non-normalized data. Textual data should be valid, NFC-normalized Unicode without C0 control characters other than HT (\t), LF (\n), and CR (\r).

# Read TSV

In [93]:
maknuune_file = "maknuune-v1.0.1.tsv"

singular_nouns = ['NOUN:MS', 'NOUN:FS']

df = pd.read_csv(maknuune_file,sep='\t')

# Stats by PoS

In [94]:
df_count = df.groupby(['ANALYSIS'])['ANALYSIS'].count()

In [95]:
df_count.loc[lambda x : x > 50].sort_values(ascending=False)

ANALYSIS
VERB:I             6345
VERB:C             6329
VERB:P             6191
NOUN:MS            4183
NOUN:FS            3050
ADJ:MS             2388
NOUN:PHRASE        2343
NOUN:P             1894
NOUN_ACT:MS         759
VERB:PHRASE         639
NOUN_PASS           305
ADJ:P               288
ADJ:PHRASE          249
NOUN_ACT:PHRASE     117
ADJ/NOUN            108
INTERJ              106
ADJ:FS               78
ADJ_COMP             77
ADV                  71
NOUN_PROP            65
Name: ANALYSIS, dtype: int64

# First: singular nouns deriving from a root and without notes 

In [96]:
# 1355 entries with notes => deal with them later
# df.loc[df['NOTES'].notnull()].count()

# 897 entries with NTWS roots => deal with them later
# df.loc[df['ROOT'] == 'NTWS'].count()

# df['LEMMA_BW'] == df['FORM_BW'] to exclude weird cases for nouns

df_singular_nouns = df.loc[(df['ROOT'] != 'NTWS') & (df['ANALYSIS'].isin(singular_nouns)) & (df['LEMMA_BW'] == df['FORM_BW']) & (df['NOTES'].isnull())]

df_singular_nouns

,ID,ROOT,ROOT_NTWS,ROOT_1,LEMMA,LEMMA_SEARCH,FORM,LEMMA_BW,FORM_BW,CAPHI++,ANALYSIS,GLOSS,GLOSS_MSA,EXAMPLE_USAGE,NOTES,SOURCE,ANNOTATOR,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1,ء.ب.د,NaN,ء,أَبَد,أبد,أَبَد,>abad,>abad,2 a b a d,NOUN:MS,eternity;forever,NaN,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN
1,2,ء.ب.ر,NaN,ء,إِبْرِة,إبرة,إِبْرِة,<iborip,<iborip,2 i b r e,NOUN:FS,needle;injection,إِبْرَة#حُقْنَة,في إِبْرِة وقعت تحت الكنب دير بالك.#أخذت ابرة ...,NaN,NaN,شهد دعباس,NaN,NaN,NaN
7,8,ء.ب.ط,NaN,ء,أَبَاط,أباط,أَبَاط,>abaAT,>abaAT,2 a b aa t.,NOUN:MS,armpit,إِبْط,ولك ليش أباطك أسود هيك؟,NaN,NaN,شهد دعباس,NaN,NaN,NaN
8,9,ء.ب.ط,NaN,ء,بَاط,باط,بَاط,baAT,baAT,b aa t.,NOUN:MS,armpit,إِبْط,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN
10,11,ء.ب.و,NaN,ء,أَب,أب,أَب,>ab,>ab,2 a b,NOUN:MS,father,أب,الله يرحمه أبوه بقى زلمة مليح وبينشد فيه الظهر,NaN,NaN,شهد دعباس,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36039,36263,ي.م.ن,NaN,ي,يَمِين,يمين,يَمِين,yamiyn,yamiyn,y a m ii n,NOUN:MS,right,يمين,روتش لك شوي لليمين,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36040,36264,ي.م.ن,NaN,ي,يَمِين,يمين,يَمِين,yamiyn,yamiyn,y a m ii n,NOUN:MS,vow;oath,NaN,احلف يَمِين إِنه مالكاش دخَّل!,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36044,36268,ي.م.ن,NaN,ي,يُمُن,يمن,يُمُن,yumun,yumun,y u m u n,NOUN:MS,blessing,نِعْمَة,ألف مبارك الشهر وكل عام وانتو بخير وان شاء الل...,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36056,36280,ي.و.م,NaN,ي,يَومِيِّة,يومية,يَومِيِّة,yawmiy~ip,yawmiy~ip,y oo m i y y e,NOUN:FS,daily_payment,دفع بنظام المياومة,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN


# Wiktionary format

In [98]:
beginning_file = """'''User:A455bcd9/Maknuune/{entry}'''
{{{{-start-}}}}"""

template = """==South Levantine Arabic==
{{{{bulk import|{{{{R:ajp:Maknuune|id={id}}}}}}}}}
{{{{ajp-root|{root}}}}}

===Pronunciation===

* {{{{ajp-IPA|{ipa}}}}}

===Noun===
{{{{ajp-noun|head={head}|g={gender}|tr={transliteration}{plural}}}}}

# {glosses}{example}

===References===
* {{{{R:ajp:Maknuune}}}}"""

end_file="{{{{-stop-}}}}"

"""
Plural:

===Noun===
{{ajp-noun|head={head}|g=p|tr={transliteration}}}
# {{plural of|ajp|{singular}|tr={singulartransliteration}|t={singulargloss}}}
# [[definition]]

"""

'\nPlural:\n\n===Noun===\n{{ajp-noun|head={head}|g=p|tr={transliteration}}}\n# {{plural of|ajp|{singular}|tr={singulartransliteration}|t={singulargloss}}}\n# [[definition]]\n\n'

# Select 10 random entries

In [99]:
# df_singular_nouns = df_singular_nouns.sample(n=1000)

# CAPHI++ to Wikt

In [100]:
for index, row in df_singular_nouns.iterrows():
    caphipp = row['CAPHI++'].strip()
    ipa = caphipp2ipa(caphipp, mode='wikt')
    # print(row['LEMMA'], '=>', caphipp, '=>', ipa)
    
# test m a w J uu d => mawjūd (موجود)
# maw(dʒ)uːd // مَوْ ُجود
print(caphipp2ipa('m a w J uu d', mode='wikt'))
# 2 i sh t aa Q => ištāʔ (اشتاق)
# ʔiʃtaː(q)
print(caphipp2ipa('2 i sh t aa Q', mode='wikt'))

mawjūd
ʔištāʔ


# Test entries

In [101]:
test_entries = [
    'أب',
    'أباط',
    'أجار',
    'أجرة',
    'أخذ',
    'ألفة',
    'إيجار',
    'باط',
    'بلع',
    'تأثير',
    'تأجير',
    'تأجيل',
    'تجارة',
    'خرقة',
    'زهاب',
    'عنصرية',
    'غواصة',
    'قذلة',
    'كماجة',
    'لقمة',
    'مهاجر',
    'مواخذة',
    'شرعية',
    'بيئة',
    'كيلو',
    'مفاد',
    'خضرة',
    'نظرة',
    'فناء',
    'إفشاء',
    'عجلة',
    'صحصحة',
    'واعظ',
]

# Save entries in Wiktionary

## Clean glosses

In [102]:
def clean_gloss(gloss):
    clean_gloss = gloss
    
    replacements = [
        ("_", " "),
        (r' +', ' ',),
        ('sb/sth', 'somebody or something'),
        ("(?i) sth", " something"),
        ("(?i) sb", " somebody"),
        (r"(?i)^sb ", "somebody "),
        (r"(?i)^sth ", "something "),
        ("(?i) some one ", " someone "),
        (r"(?i)^a type of ", ""),
        (r"^see phrase", ""),
        (r"(?i)^a ", ""),
        (r"(?i)^an ", ""),
        (r"(?i)^The ", ""),
        (r"(?i)^It is what ", "what "),
        (r"(?i)^It is (a|the) ", ""),      
    ]
    
    for replacement in replacements:
        clean_gloss = re.sub(replacement[0], replacement[1], clean_gloss)
    
    return clean_gloss.strip().strip('.')

In [103]:
def test_clean_gloss():
    to_test = [
        ('a_type_of_jar_for_keeping_water_cold', 'jar for keeping water cold'),
        ('see_phrase', ''),
        ('ceramic work.', 'ceramic work'),
        ('sb_who_eats_food_off_the_ground', 'somebody who eats food off the ground'),
        ('a small ball', 'small ball'),
        ('a single person', 'single person'),
        ('a narrow path', 'narrow path'),
        ('a fat person', 'fat person'),
        ('sb/sth', 'somebody or something'),
        ('a difficult  situation', 'difficult situation'),
        ('the grated food', 'grated food'),
        ('the beautiful woman', 'beautiful woman'),
        ('a_stone_inside_a_hole', 'stone inside a hole'),
        ('an individual.', 'individual'),
        ("the_state_of_holding_something_over_sb's_head", "state of holding something over somebody's head"),
        ('a_vessel_that_is_usually_of_brass_for_kohl.', 'vessel that is usually of brass for kohl'),
        ('It_is_what_is_placed_on_the_back_of_the_walking_animal_for_the_purpose_of_riding_on_it', 'what is placed on the back of the walking animal for the purpose of riding on it'),
        ('It_is_the_ritual_animal_sacrifice_of_a_livestock_animal_in_the_first_three_days_of_the_funeral_that_is_believed_to_give_solace_to_the_deceased_person', 'ritual animal sacrifice of a livestock animal in the first three days of the funeral that is believed to give solace to the deceased person'),
        ('A_thin_layer_of_freezing_water_in_the_morning,_due_to_lower_temperatures.', 'thin layer of freezing water in the morning, due to lower temperatures'),
        ('It_is_a_traditional_type_of_dessert_that_is_made_of_dough,_water_and_molasses._It_is_usually_eaten_with_dried_figs.', 'traditional type of dessert that is made of dough, water and molasses. It is usually eaten with dried figs'),
        ('sth_similar_to_a_container_that_is_made_by_sewing_several_"7_u_s._u_r"_together._The_farmers_usually_keep_wheat_in_it.', 'something similar to a container that is made by sewing several "7 u s. u r" together. The farmers usually keep wheat in it'),
        ('the_person_whose_job_is_to_tell_stories_to_the_people', 'person whose job is to tell stories to the people'),
    ]
    for unit_test in to_test:
        if clean_gloss(unit_test[0]) != unit_test[1]:
            raise ValueError('Unit test failed:', clean_gloss(unit_test[0]), unit_test[1])
            
    return 'clean_gloss(): OK'

test_clean_gloss()

'clean_gloss(): OK'

In [105]:
site = pywikibot.Site()
i = 0

for index, row in df_singular_nouns.iterrows():
    # remove _[auto] from entries generated automatically from other entries
    clean_gloss_row = row['GLOSS'].replace('_[auto]', '').strip()
    # filter out more complex glosses
    entry_values = {}
    entry_values['id'] = row['ID']
    entry_values['entry'] = row['LEMMA_SEARCH'].strip()
    entry_values['root'] = row['ROOT'].strip().replace('.', ' ')
    entry_values['head'] = row['FORM'].strip()
    entry_values['gender'] = 'f' if row['ANALYSIS'] == 'NOUN:FS' else 'm'

    caphipp = row['CAPHI++'].strip()
    entry_values['transliteration'] = caphipp2ipa(caphipp, mode='wikt')
    entry_values['ipa'] = entry_values['transliteration']
    
    entry_values['plural'] = ''

    entry_values['example'] = ''
    if not pd.isna(row['EXAMPLE_USAGE']):
        example = row['EXAMPLE_USAGE'].strip()
        # Fill example if not null
        if example != '':
            entry_values['example'] = '\n#: {{{{ux|ajp|{example}}}}}'.format(example=example)

    # Separate glosses
    glosses = set(clean_gloss_row.split(';'))
        
    to_save = False
    if '_' not in clean_gloss_row and ' ' not in clean_gloss_row:
        entry_values['glosses'] = '[[' + ']], [['.join(glosses) + ']]'
    else:
        # Could check whether English n-grams (glosses) are already in the Wiktionary and link them
        # https://en.wiktionary.org/w/api.php?action=query&titles=test&format=json

        # add link for simple glosses in the list
        entry_values['glosses'] = ', '.join([clean_gloss(x) for x in glosses])
        print(entry_values['id'], entry_values['glosses'])
        to_save = False

    new_entry = template.format(**entry_values)

    page_name = u"User:A455bcd9/Maknuune/{entry}".format(entry=entry_values['entry'])
    page = pywikibot.Page(site, page_name)

    page.text = new_entry
    if to_save:
        print(new_entry)
        # page.save(u"Entries with multiple glosses and examples")
        i += 1    
            
    if i > 20:
        break

print('end')

28 father Pope, daddy
30 possessor of, father of, having a particular quality
36 furnishing (a place)
66 Hasanat (Credit for good deeds, which Allah weighs up against one's bad deeds at the final judgement after death)
107 taking something
119 Day of the Judgment
122 stable (horses)
123 obligation, in the form of money or possessions paid by the groom, to the bride at the time of Islamic divorce
161 have the spirit of brotherhood (usually between a man and a woman)
177 brother, brother of
206 Adhan, call for prayer
207 Adhan, call for prayer
208 caller of prayer
255 territory, land, land plot
256 territory, land, land plot
283 It is an alternative form of the cloak made of white linen fabric or pure cotton
301 state of being a teacher, professorship
382 0.5 pound
386 getting used to something, acclimatizing oneself to something
456 authoring, making something up, composing, lying
496 noble person, prince
534 Imam is an Islamic leadership position. Imam is usually the person who leads t

2965 speaking fluently
2969 passage
3052 salivating, spitting a lot
3109 philanthropic institution, free eating-place
3122 insesitive, state of being thick-skinned
3220 large can
3253 metal bowl that is painted with colours
3297 dim-witted / weak (student), goat
3327 appointing somebody, confirming something
3328 proving something
3329 state of bing proven
3377 raising awareness, educating
3457 ploughing the land for the second time
3737 state of writhing in pain, complaining about pain
3742 state of writhing in pain, complaining about pain
3796 being forced to go, pulling something repeatedly
3798 black olives that have been collected from the ground
3807 state of being very impatient
3808 
3834 for the sole reason that, for no other reason but, mere, nothing but
3841 floor wiper
3870 slaughtered sheep, calves or cows that the attendees usually bring as gifts to the groom's family on the Wedding day
3884 long sleever blouse
3885 shirt made out of wool
3887 scandal in public
3888 scand

6206 money box
6207 chicken's crop
6209 alveoli, air sac
6263 writ, memorandum of a case reported to the police
6265 going through the death throes
6288 piece of fabric that protects the people in the tent from the sun rays
6293 hugging somebody excessively
6306 collector and seller of firewood
6320 engagement party in some villages
6321 price increase
6322 White transparent silk called "Al-Ayyobal" and "Al-Aghbani", which is white with striped golden scissors and worn with a golden headband on holidays
6337 dampening somebody's spirit, damage, damage to somebody's morale
6369 moving a lot back and forth
6379 hole digger (auger), Zucchini corer
6412 Quraanic school
6413 keeping something, memorization
6443 removing (the unwanted body hair)
6515 can that contains nasal powder to improve inhale
6558 house garden, backyard
6583 court, court hearing
6585 judicial proceeding, legal prosecution
6616 person whose job is to tell stories to the people, storyteller
6638 Fenugreek, Fenugreek dess

8573 monetary gifts
8612 human, person, human being
8630 piece of land that lies between two mountains
8634 olive stalk (it is used to close the large bag made of coarse cloth where olives are kept)
8650 bag for food that is hanged on the donkey's or horse's neck
8652 small bag, in which the animal's food is placed in it. It is placed on the neck of the animals during plowing or resting
8688 ring shaped pieces of bread or biscuits that are made from many ingredients. The flour, sugar, olive oil, (ghee optional), milk are mixed together. Anise, sesame and black cumin are then added to the mixture. The dough is left to rest for one hour. After that, they strech it and leave it again to rest in order to have a better shape. It is made into dough balls that are placed and flattened into a large baking tray
8712 golden coin that is attached to the "z a n n aa q", i.e. (a silver chain (like a necklace) where other small chains with silver coins are attached to it)
8754 trash
8755 hen house
8

11535 deploying (troops), sending, broadcast, transmission, deploying (trops)
11545 formal way, formalism
11558 rein, rope simpler than the bridle, which is made of fine ropes, and it connects with a long rope that the rider holds, to direct or stop the donkey
11577 raising awareness, rationalization
11586 machine gun
11588 spray, lavishing money and gifts on somebody, spraying the crops with pesticides, sprinkle
11594 traditional dish that is made of lentils, rice and yoghurt
11598 throwing liquid at somebody, asking somebody so many question
11613 noseband (in a bridle)
11614 lot of writing
11615 ring shaped pieces of bread or biscuits that are made from many ingredients. The flour, sugar, olive oil, (ghee optional), milk are mixed together. Anise, sesame and black cumin are then added to the mixture. The dough is left to rest for one hour. After that, it is made into dough balls that are placed and flattened into a large baking tray
11647 dental filling, stuffing
11650 pickled olive

...

24072 state of being worldly-wise, eloquence
24082 fart, clove (of garlic)
24083 thorough scrutiny, repeated fart
24095 biting down on seeds, dissection or nitpicking
24101 style or cut (clothes)
24105 door hinge
24136 situation where somebody contradicts himself
24168 removing soap and foam from, pouring somebody's heart out, loosening something
24173 favour, good action
24186 free time
24202 breaking of the (Ramadan) fast
24203 pie
24217 any stinky or dirty thing, carrion
24228 reacting towards something and jumping up and down hesterically
24255 state of being too horrible
24263 searching for something, rummaging through
24295 yelling at somebody, telling somebody off, explosion
24310 state of being messy and untidy
24339 making something up
24357 biting into something
24369 Palestinian tradition where people visit the bride's house and bring her some utensils that she will use in her kitchen. This visit is usually made after one week of the wedding ceremony
24418 egg incubator, pla

26176 making somebody or something unable to move because of cold
26185 barley stalk
26226 tuck (in a garment or any other clothes)
26227 attracting somebody or something
26251 drop bottle
26253 sugar syrup
26265 clay pot that is somewhat similar to the deep dish. It was used to curdle the milk into yoghurt before it was sold
26268 Islamic sarcifice (a sheep that is killed according to the Islamic rules and teachinngs) for the sake of God that is done when the cattle of sheep is sick
26300 piece, white headband that is worn by women under their headscarves
26302 headquarter to the high Palestinian authority leadership
26303 dough cutter, dough cutting tool
26306 small warehouse
26309 facial feature
26312 passageway between two land plots
26355 picking
26356 dessert usually made in Ramadan consisting of the dough of Qatayef (consisting of flour, semolina, water, milk, and yeast) that is stuffed with sweetened cheese, nuts, or with cream
26357 process of picking
26358 basket that is made

28238 black olives
28263 problem, polyp, social problem, knot
28272 woolen baby shoes
28276 old house wall
28328 decorative silk (fabric)
28356 wearing a lot of clothes either because it is cold, or because somebody wants to get dressed modestly, covering oneself entirely, including the head. However, the person leavs his eyes uncovered in order to see things clearly
28364 wearing a lot of clothes either because it is cold, or because somebody wants to get dressed modestly, covering oneself entirely, including the head. However, the person leavs his eyes uncovered in order to see things clearly
28382 conclusion become complete, perfection, become perfect, be concluded, completeness
28388 sleeve of the garment
28392 mask, face covering
28429 broom, vacuum cleaner
28431 broom, vacuum cleaner
28433 broom, vacuum cleaner
28444 huddling together for warmth
28450 famous dessert made from konafa dough, which is vermicelli in the form of long filaments added with margarine, sugar syrup, red dy

30693 maimed, deformed person, somebody who looks like a monkey
30703 flattening the dough evenly, stretching the dough
30712 long thick rod with a sharp nail on top of it that the peasant uses to urge the animal to move, and at the rear is a sharp flat piece of iron to remove the clay if it is stuck with the rail
30717 pot-holder, net or trap
30718 bridal bouquet
30720 state of holding something against somebody
30744 sailmaker's needle
30746 trees bear good fruits (especially olive trees)
30766 Palestinian omelette Fritters with cauliflower "Mshat"
30812 lower part of the chicken thigh (drumstick)
30814 silk thread
30831 sucking on something
30855 water bottle
30856 hole in the ground that is covered with straw, and that is used to store grains like wheat
30864 state of enlogating a sound, state of stretching
30883 person who tarries or lingers
30906 plucking machine
30921 washer, washing machine
30938 scrutiny, close examination
30939 pans, any of the kitchen utensils, pot
30948 kis

33241 special type of food that is served by the relatives to the deceased's family after the burial
33242 missing part
33262 breaking (a promise), reneging on
33312 moving from one place to another
33313 conveying a message or recounting a story from one place to another
33346 nibbling at something, eating a few bites
33353 eating small bites or small quantites of food from all the dishes, nibbling at something, getting a bite to eat
33354 eating small bites or small quantites of food from all the dishes, nibbling at something, getting a bite to eat
33405 state of being peevish, ill-tempered, moody
33445 embarrassing situation, setback
33465 joke, raising an issue
33482 nail polish
33506 teasing somebody
33510 dessert consisting of: semolina, maragrine, sugar, butter, yoghurt, flour, baking powder, blossom water and sugar syrup
33551 piece of gossip
33557 state of being itchy, tingling
33593 badger, nagging, making troubles
33613 saying "ugh", beating somebody severely
33627 biting in

In [ ]:
"""
issues
"""